<a href="https://colab.research.google.com/github/7kanak/Leap_Gesture_Recognition/blob/master/tensorflow_resnet50_99_95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import PIL
from IPython.display import clear_output

In [0]:
tf.enable_eager_execution()

In [0]:
os.chdir("/content/drive/My Drive/Projects/Gesturec/data/leapGestRecog/")

In [0]:
classes = os.listdir()

In [0]:
classes

['01_palm',
 '09_c',
 '10_down',
 '08_palm_moved',
 '04_fist_moved',
 '07_ok',
 '02_l',
 '05_thumb',
 '03_fist',
 '06_index']

In [0]:
int_classes = list(range(len(classes)))

In [0]:
int_classes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [0]:
filenames = []
labels = []
for idx,gesture in enumerate(classes):
  tmp = os.listdir(gesture)
  tmp = [gesture+"/"+i for i in tmp]
  filenames = filenames + tmp
  labels = labels + [idx]*(len(tmp))

In [0]:
filenames[-5:],labels[-5:]

(['06_index/frame_04_06_0196.png',
  '06_index/frame_04_06_0197.png',
  '06_index/frame_04_06_0198.png',
  '06_index/frame_05_06_0007.png',
  '06_index/frame_04_06_0199.png'],
 [9, 9, 9, 9, 9])

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_ds, val_ds, label_train, label_val = train_test_split(filenames, labels, test_size=0.2, random_state=42)

In [0]:
data_length = len(filenames);data_length

20000

In [0]:
image_size = 224 # All images will be resized to 160x160
batch_size = 32

In [0]:
def tfdata_generator(filenames, labels, is_training=True, batch_size=32):
    '''Construct a data generator using tf.Dataset'''

    def parse_function(filename, label):
      image_string = tf.read_file(filename)
  
      # Don't use tf.image.decode_image, or the output shape will be undefined
      image = tf.image.decode_png(image_string, channels=3)
  
      # This will convert to float values in [0, 1]
      image = tf.image.convert_image_dtype(image, tf.float32)
  
      image = tf.image.resize_images(image, [image_size, image_size])
      
      label = tf.one_hot(tf.cast(label, tf.uint8), 10)
        
      return image, label
    
    def train_preprocess(image, label):
      image = tf.image.random_flip_left_right(image)
      image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
      image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    
        # Make sure the image is still in [0, 1]
      image = tf.clip_by_value(image, 0.0, 1.0)
    
      return image, label

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    if is_training:
        dataset = dataset.shuffle(len(filenames))  # depends on sample size
        
    dataset = dataset.map(parse_function, num_parallel_calls=4)
    dataset = dataset.map(train_preprocess, num_parallel_calls=4)
    dataset = dataset.batch(batch_size,drop_remainder=True if is_training else False)
    
    dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)

    return dataset

In [0]:
training_set = tfdata_generator(train_ds, label_train)
testing_set  = tfdata_generator(val_ds, label_val, is_training=False)

In [0]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,include_top=False,  weights='imagenet')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
base_model.trainable = False

In [0]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(10, activation='softmax')
])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
len(model.trainable_variables)

2

In [0]:
model.compile('adam', 'categorical_crossentropy', metrics=['acc'])

In [0]:
model.fit(
      training_set.make_one_shot_iterator(),
      steps_per_epoch=16000 // 32,
      epochs=5,
      validation_data=testing_set.make_one_shot_iterator(),
      validation_steps=4000 //32,
      verbose = 1)

Epoch 1/5
500/500 [==============================] - 202s 404ms/step - loss: 0.2670 - acc: 0.9458 - val_loss: 3.1156 - val_acc: 0.1357
Epoch 2/5
500/500 [==============================] - 199s 397ms/step - loss: 0.0412 - acc: 0.9953 - val_loss: 0.5274 - val_acc: 0.8395
Epoch 3/5
500/500 [==============================] - 199s 398ms/step - loss: 0.0231 - acc: 0.9969 - val_loss: 0.0359 - val_acc: 0.9945
Epoch 4/5
500/500 [==============================] - 200s 399ms/step - loss: 0.0153 - acc: 0.9978 - val_loss: 0.0071 - val_acc: 0.9995
Epoch 5/5
500/500 [==============================] - 198s 396ms/step - loss: 0.0108 - acc: 0.9987 - val_loss: 0.0055 - val_acc: 0.9995


In [0]:
model.save_weights('./checkpoints/leap_resnet50')



Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


In [0]:
checkpoint_dir = '../../data/'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tf.train.Checkpoint(optimizer=optimizer,
                           model=model,
                           optimizer_step=tf.train.get_or_create_global_step())

root.save(checkpoint_prefix)

'../../data/ckpt-1'

In [0]:
for a,b in val_dataset.take(-1):
  prediction = model.predict(a,steps=1)
  res = tf.argmax(tf.nn.softmax(prediction),axis=1)

In [0]:
res.numpy() == b.numpy()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [0]:
b

<tf.Tensor: id=96335, shape=(32,), dtype=int32, numpy=
array([3, 8, 8, 3, 4, 8, 0, 1, 9, 2, 9, 5, 1, 4, 8, 4, 4, 5, 0, 5, 7, 4,
       9, 6, 2, 7, 6, 2, 5, 9, 5, 5], dtype=int32)>

In [0]:
??model.predict